<a href="https://colab.research.google.com/github/KYOH95/Database_Studies/blob/main/ai_sc12x.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 - Bayesian

총 다섯 단원에 대하여 20문항을 출제한 시험에서 한 학생의 결과는 정답 12문항, 오답 8문항입니다. </br>
이때 맞은 문항 중 1단원은 3문항이고 틀린 문항 중 1단원은 1문항입니다. 이 학생이 1단원에 대해서 맞출 확률을 **반올림하여 소수점 둘째 자리**까지 구하시오. 

In [119]:
p = 12/20
np = 8/20
correct_1 = 3/12
wrong_1 = 1/8

numerator = correct_1 * p
denominator = (correct_1 * p) + (wrong_1 * np)
posterior = numerator / denominator

round(posterior,2)

0.75

# Part 2 - 가설검정

이번 Part에서 여러분은 가상의 마케팅 데이터를 활용하여 가설검정을 진행하시게 됩니다. 우리가 사용할 $\chi^2$ - test는 어떤 경우에 사용하는 가설 검정 방법인지 알아봅시다.

---

- one sample $\chi^2$ - test (적합도 검정)
  > **주어진 하나의 범주형 변수가 특정 예상되는 분포와 동일한 분포를 나타내는지** 에 대한 가설 검정 방법

  - 귀무가설($H_0$): 주어진 변수 X의 관측분포와 특정 예상 분포가 동일하다.
  - 대립가설($H_a$): 주어진 변수 X의 관측분포와 특정 예상 분포가 다르다.

- two sample $\chi^2$ - test (독립성 검정)
  > **두 개의 범주형 변수가 서로 연관되어 있는지 여부**에 대한 가설 검정 방법

  - 귀무가설($H_0$): 변수 X와 Y는 서로 독립이다. (연관 없음)
  - 대립가설($H_a$): 변수 X와 Y는 서로 독립이 아니다. (연관 있음)

</br>

위의 가설검정을 아래의 데이터셋을 사용하여 진행해봅니다.

## 데이터셋 정보 :   
이 데이터셋은 알뜰 구매를 도와주는 XYZ사의 마케팅 정보를 다룹니다. 개인 정보는 보호를 위해 제거 되어 있습니다.

**Attribute Information:**
> 2,240 customers of XYZ company:

- ID - 고객의 고유값
- Year_Birth - 고객의 출생연도
- Education - 고객의 최종학력
- Marital_Status - 고객의 혼인여부
- Country - 고객의 국적
- Income - 고객의 연간 수입
- Kidhome - 자녀가 미취학아동인지 여부 (1 - yes, 0 - no)
- Teenhome - 자녀가 10대 청소년인지 여부 (1 - yes, 0 - no)
- Dt_Customer - 서비스 가입 시기
- Recency - 마지막 구매 시기 (일)
- MntWines - 최근 2년간 구매한 와인 구매 총액
- MntFruits- 최근 2년간 구매한 과일 구매 총액
- MntMeatProducts - 최근 2년간 구매한 정육 구매 총액
- MntFishProducts - 최근 2년간 구매한 어류 구매 총액
- MntSweetProducts - 최근 2년간 구매한 과자 구매 총액
- MntGoldProds - 최근 2년간 구매한 황금 구매 총액
- NumDealsPurchases - 할인을 통해 제품을 구매한 횟수
- NumWebPurchases - 회사의 웹사이트를 통해 구매한 횟수
- NumStorePurchases - 가게에서 직접 구매한 횟수
- NumWebVisitsMonth - 회사의 웹사이트를 방문한 횟수
- Complain - 고객의 최근 2년간 회사에 대한 클레임 여부 (1 - yes, 0 - no)


## 준비사항

>📌 본격적인 가설검정에 들어가기에 앞서 다음의 사항을 진행해주세요.

1. 데이터셋을 업로드 합니다
2. 결측치가 있는지 확인 후, 있다면 제거합니다


#### 1. 데이터셋을 업로드 합니다

In [120]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [121]:
# from google.colab import files
# files.upload()

In [122]:
df = pd.read_csv('sc12x_dataset.csv')
df.shape

(2240, 22)

#### 2. 결측치가 있는지 확인 후, 있다면 제거합니다

In [123]:
df.isnull().sum()

Unnamed: 0            0
ID                    0
Year_Birth            0
Country               0
Education             0
Marital_Status        0
 Income              24
Kidhome               0
Teenhome              0
Dt_Customer           0
Recency               0
MntWines              0
MntFruits             0
MntMeatProducts       0
MntFishProducts       0
MntSweetProducts      0
MntGoldProds          0
NumDealsPurchases     0
NumWebPurchases       0
NumStorePurchases     0
NumWebVisitsMonth     0
Complain              0
dtype: int64

In [124]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         2240 non-null   int64 
 1   ID                 2240 non-null   int64 
 2   Year_Birth         2240 non-null   int64 
 3   Country            2240 non-null   object
 4   Education          2240 non-null   object
 5   Marital_Status     2240 non-null   object
 6    Income            2216 non-null   object
 7   Kidhome            2240 non-null   int64 
 8   Teenhome           2240 non-null   int64 
 9   Dt_Customer        2240 non-null   object
 10  Recency            2240 non-null   int64 
 11  MntWines           2240 non-null   int64 
 12  MntFruits          2240 non-null   int64 
 13  MntMeatProducts    2240 non-null   int64 
 14  MntFishProducts    2240 non-null   int64 
 15  MntSweetProducts   2240 non-null   int64 
 16  MntGoldProds       2240 non-null   int64 


In [125]:
df = df.dropna(axis=0)

In [126]:
df = df.drop(df.columns[[0]], axis=1)
df

ID  Year_Birth Country   Education Marital_Status      Income   \
0      1826        1970      SP  Graduation       Divorced  $84,835.00    
1         1        1961      CA  Graduation         Single  $57,091.00    
2     10476        1958      US  Graduation        Married  $67,267.00    
3      1386        1967     AUS  Graduation       Together  $32,474.00    
4      5371        1989      SP  Graduation         Single  $21,474.00    
...     ...         ...     ...         ...            ...          ...   
2235  10142        1976      US         PhD       Divorced  $66,476.00    
2236   5263        1977      SP    2n Cycle        Married  $31,056.00    
2237     22        1976      SP  Graduation       Divorced  $46,310.00    
2238    528        1978     IND  Graduation        Married  $65,819.00    
2239   4070        1969      CA         PhD        Married  $94,871.00    

      Kidhome  Teenhome Dt_Customer  Recency  ...  MntFruits  MntMeatProducts  \
0           0         0     6/16/14        0  ...        104              379   
1           0         0     6/15/14        0  ...          5               64   
2           0         1     5/13/14        0  ...         11               59   
3           1         1     5/11/14        0  ...          0                1   
4           1         0      4/8/14        0  ...         16               24   
...       ...       ...         ...      ...  ...        ...              ...   
2235        0         1      3/7/13       99  ...         18              126   
2236        1         0     1/22/13       99  ...         10               13   
2237        1         0     12/3/12       99  ...          2               88   
2238        0         0    11/29/12       99  ...         38              701   
2239        0         2      9/1/12       99  ...         24              553   

      MntFishProducts  MntSweetProducts  MntGoldProds  NumDealsPurchases  \
0                 111               189           218                  1   
1                   7                 0            37                  1   
2                  15                 2            30                  1   
3                   0                 0             0                  1   
4                  11                 0            34                  2   
...               ...               ...           ...                ...   
2235               47                48            78                  2   
2236                3                 8            16                  1   
2237               15                 5            14                  2   
2238              149               165            63                  1   
2239              188                 0           144                  1   

      NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  Complain  
0                   4                  6                  1         0  
1                   7                  7                  5         0  
2                   3                  5                  2         0  
3                   1                  2                  7         0  
4                   3                  2                  7         0  
...               ...                ...                ...       ...  
2235                5                 11                  4         0  
2236                1                  3                  8         0  
2237                6                  5                  8         0  
2238                5                 10                  3         0  
2239                8                  4                  7         0  

[2216 rows x 21 columns]

In [127]:
df = df.reset_index(level=None, drop=True, inplace=False, col_level=0, col_fill='')
df

ID  Year_Birth Country   Education Marital_Status      Income   \
0      1826        1970      SP  Graduation       Divorced  $84,835.00    
1         1        1961      CA  Graduation         Single  $57,091.00    
2     10476        1958      US  Graduation        Married  $67,267.00    
3      1386        1967     AUS  Graduation       Together  $32,474.00    
4      5371        1989      SP  Graduation         Single  $21,474.00    
...     ...         ...     ...         ...            ...          ...   
2211  10142        1976      US         PhD       Divorced  $66,476.00    
2212   5263        1977      SP    2n Cycle        Married  $31,056.00    
2213     22        1976      SP  Graduation       Divorced  $46,310.00    
2214    528        1978     IND  Graduation        Married  $65,819.00    
2215   4070        1969      CA         PhD        Married  $94,871.00    

      Kidhome  Teenhome Dt_Customer  Recency  ...  MntFruits  MntMeatProducts  \
0           0         0     6/16/14        0  ...        104              379   
1           0         0     6/15/14        0  ...          5               64   
2           0         1     5/13/14        0  ...         11               59   
3           1         1     5/11/14        0  ...          0                1   
4           1         0      4/8/14        0  ...         16               24   
...       ...       ...         ...      ...  ...        ...              ...   
2211        0         1      3/7/13       99  ...         18              126   
2212        1         0     1/22/13       99  ...         10               13   
2213        1         0     12/3/12       99  ...          2               88   
2214        0         0    11/29/12       99  ...         38              701   
2215        0         2      9/1/12       99  ...         24              553   

      MntFishProducts  MntSweetProducts  MntGoldProds  NumDealsPurchases  \
0                 111               189           218                  1   
1                   7                 0            37                  1   
2                  15                 2            30                  1   
3                   0                 0             0                  1   
4                  11                 0            34                  2   
...               ...               ...           ...                ...   
2211               47                48            78                  2   
2212                3                 8            16                  1   
2213               15                 5            14                  2   
2214              149               165            63                  1   
2215              188                 0           144                  1   

      NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  Complain  
0                   4                  6                  1         0  
1                   7                  7                  5         0  
2                   3                  5                  2         0  
3                   1                  2                  7         0  
4                   3                  2                  7         0  
...               ...                ...                ...       ...  
2211                5                 11                  4         0  
2212                1                  3                  8         0  
2213                6                  5                  8         0  
2214                5                 10                  3         0  
2215                8                  4                  7         0  

[2216 rows x 21 columns]

### 1) $\chi^2$ 가설 검정을 시행합니다

#### 1-1) one sample $\chi^2$ - test 

> XYZ사는 다국적 기업으로 다양한 나라에 고객층을 확보하고 있으며, 5월을 맞아 **기혼자를 대상으로 한 이벤트를 기획**하려고 합니다. 이때, 우리는 모든 국가를 대상으로 이벤트를 진행하는 것이 효율적인지 $\chi^2$ - test를 통해 확인해보려 합니다. 만약 국적별 기혼자의 빈도가 비슷하다면, XYZ사는 모든 국가를 대상으로 이벤트를 진행하는 것을 고려할 것이고, 그렇지 않다면, 어떤 국적의 고객이 더 많은지 파악하는 추가 분석이 필요할 것입니다.

지금부터 **기혼자의 빈도가 모든 나라에 대해 균일한지** 가설 검정을 해봅시다.

---

[공식문서](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html)를 참고하여 아래에 명시된 가설을 검정해봅시다. ($\alpha$ = 0.05)
- 귀무가설($H_0$): 고객의 국적(Country)별 기혼자(Married)의 빈도가 균일하게 분포되어 있다.
- 대립가설($H_a$): 고객의 국적(Country)별 기혼자(Married)의 빈도가 균일하게 분포되어 있지 않다.

> **Hint:** categorical feature 간의 "cross tab"을 생성하여 탐색해 보세요 (aka [contingency tables](https://en.wikipedia.org/wiki/Contingency_table)). 그리고 $\chi^2 test$를 적용해 보세요. [pandas.crosstab](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html)을 사용하면 contingency table 을 생성할 수 있습니다.

In [128]:
from scipy.stats import chisquare

# 이 곳에 답안을 작성하십시오

In [129]:
condition = df["Marital_Status"] == "Married"
df[condition]

ID  Year_Birth Country   Education Marital_Status      Income   \
2     10476        1958      US  Graduation        Married  $67,267.00    
6      4073        1954     GER    2n Cycle        Married  $63,564.00    
8      4047        1954      US         PhD        Married  $65,324.00    
9      9477        1954     IND         PhD        Married  $65,324.00    
10     2079        1947      US    2n Cycle        Married  $81,044.00    
...     ...         ...     ...         ...            ...          ...   
2205   2106        1974      SP    2n Cycle        Married  $20,130.00    
2206   3363        1974      SP    2n Cycle        Married  $20,130.00    
2212   5263        1977      SP    2n Cycle        Married  $31,056.00    
2214    528        1978     IND  Graduation        Married  $65,819.00    
2215   4070        1969      CA         PhD        Married  $94,871.00    

      Kidhome  Teenhome Dt_Customer  Recency  ...  MntFruits  MntMeatProducts  \
2           0         1     5/13/14        0  ...         11               59   
6           0         0     1/29/14        0  ...         80              252   
8           0         1     1/11/14        0  ...          0              102   
9           0         1     1/11/14        0  ...          0              102   
10          0         0    12/27/13        0  ...         26              535   
...       ...       ...         ...      ...  ...        ...              ...   
2205        0         0     3/17/14       99  ...          6                3   
2206        0         0     3/17/14       99  ...          6                3   
2212        1         0     1/22/13       99  ...         10               13   
2214        0         0    11/29/12       99  ...         38              701   
2215        0         2      9/1/12       99  ...         24              553   

      MntFishProducts  MntSweetProducts  MntGoldProds  NumDealsPurchases  \
2                  15                 2            30                  1   
6                  15                34            65                  1   
8                  21                32             5                  3   
9                  21                32             5                  3   
10                 73                98            26                  1   
...               ...               ...           ...                ...   
2205                7                 6            12                  1   
2206                7                 6            12                  1   
2212                3                 8            16                  1   
2214              149               165            63                  1   
2215              188                 0           144                  1   

      NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  Complain  
2                   3                  5                  2         0  
6                  10                  7                  6         0  
8                   6                  9                  4         0  
9                   6                  9                  4         0  
10                  5                 10                  1         0  
...               ...                ...                ...       ...  
2205                1                  3                  8         0  
2206                1                  3                  8         0  
2212                1                  3                  8         0  
2214                5                 10                  3         0  
2215                8                  4                  7         0  

[857 rows x 21 columns]

In [130]:
df[condition].shape[0]

857

In [131]:
df["Country"].nunique()

8

In [132]:
#국적별 빈도가 균일하다고 가정하는 것을 기대 분포라고 합니다. 
#우리의 데이터에서 기대 분포를 만든 뒤, 이에 해당하는 빈도값을 소숫점 셋째 자리까지 구하시오.
expected = (df[condition].shape[0]) / (df["Country"].nunique())
expected

107.125

In [133]:
df_clean = df[["Country","Marital_Status"]]
df_clean

Country Marital_Status
0         SP       Divorced
1         CA         Single
2         US        Married
3        AUS       Together
4         SP         Single
...      ...            ...
2211      US       Divorced
2212      SP        Married
2213      SP       Divorced
2214     IND        Married
2215      CA        Married

[2216 rows x 2 columns]

In [134]:
contingency_table = pd.crosstab(index = [df_clean.Marital_Status], columns = df_clean.Country)
contingency_table

Country         AUS   CA  GER  IND  ME   SA   SP  US
Marital_Status                                      
Absurd            1    1    0    0   0    0    0   0
Alone             0    3    0    0   0    0    0   0
Divorced         20   27    6   10   0   47  106  16
Married          66  104   56   57   0  106  433  35
Single           18   58   18   37   2   70  237  31
Together         37   66   32   38   1  102  274  23
Widow             5    6    4    4   0   12   43   2
YOLO              0    1    0    1   0    0    0   0

In [135]:
contingency_table.loc["Expected_Distribution"] = expected #107.25
contingency_table

Country                    AUS       CA      GER      IND       ME       SA  \
Marital_Status                                                                
Absurd                   1.000    1.000    0.000    0.000    0.000    0.000   
Alone                    0.000    3.000    0.000    0.000    0.000    0.000   
Divorced                20.000   27.000    6.000   10.000    0.000   47.000   
Married                 66.000  104.000   56.000   57.000    0.000  106.000   
Single                  18.000   58.000   18.000   37.000    2.000   70.000   
Together                37.000   66.000   32.000   38.000    1.000  102.000   
Widow                    5.000    6.000    4.000    4.000    0.000   12.000   
YOLO                     0.000    1.000    0.000    1.000    0.000    0.000   
Expected_Distribution  107.125  107.125  107.125  107.125  107.125  107.125   

Country                     SP       US  
Marital_Status                           
Absurd                   0.000    0.000  
Alone                    0.000    0.000  
Divorced               106.000   16.000  
Married                433.000   35.000  
Single                 237.000   31.000  
Together               274.000   23.000  
Widow                   43.000    2.000  
YOLO                     0.000    0.000  
Expected_Distribution  107.125  107.125

In [136]:
contingency_table_clean = contingency_table.loc[['Married','Expected_Distribution']]
contingency_table_clean

Country                    AUS       CA      GER      IND       ME       SA  \
Marital_Status                                                                
Married                 66.000  104.000   56.000   57.000    0.000  106.000   
Expected_Distribution  107.125  107.125  107.125  107.125  107.125  107.125   

Country                     SP       US  
Marital_Status                           
Married                433.000   35.000  
Expected_Distribution  107.125  107.125

In [137]:
obs = contingency_table_clean.iloc[0]
exp = contingency_table_clean.iloc[1]

In [138]:
chisquare(f_obs = obs,f_exp = exp, axis=None)

Power_divergenceResult(statistic=1210.7432905484247, pvalue=3.355120916487552e-257)

In [139]:
chisquare(contingency_table.loc["Married"])

Power_divergenceResult(statistic=1210.7432905484247, pvalue=3.355120916487552e-257)

#### 1-2) P-value 해석
위의 가설 검정 결과로 반환된 P-value를 해석하여 설명해봅니다.

**\*\* `답안 작성은 이곳에 하시면 됩니다` \*\***

p-value 값이 3.355e-257로 0.05 보다 현저히 낮고 0에 가까운것을 확인 할 수 있다. 귀무가설인 "고객의 국적(Country)별 기혼자(Married)의 빈도가 균일하게 분포되어 있다."를 기각하고 대립가설을 채택해야 합니다.

# 🔥 Advanced Goals:
> SC에서 `3점`을 얻고 싶으시다면 아래의 문제를 <u>**모두**</u> 풀어주세요. 다만 `2점`만으로도 충분하니, 시간과 여유가 있는 분들만 도전해보시기 바랍니다. 


⚠️ 도전문제를 해결하더라도 기본문제에서 오류가 있다면 `1점`입니다.

## Part 1:

> 우리는 기본문제에서 사용한 마케팅 데이터셋을 그대로 사용할 것입니다.

아래의 특성들을 만드세요

1. **TotalMntProducts** - 최근 2년간 구매한 모든 제품 가격들을 합한 값을 가진 계산한 특성입니다

2. **TotalNumPurchases** - 구매횟수를 모두 합친 값을 가진 특성입니다

3. **DaysSinceJoined** - 오늘 날짜를 기준으로 가입한 기간을 보여주는 특성입니다 

4. **VIP** - 누적 구매 금액이 $2,000 이상을 넘고, 오늘을 기준으로 가입기간이 5년 이상이 된다면 1, 아니면 0의 값을 가진 칼럼을 만듭니다

In [140]:
df.isnull().sum()

ID                   0
Year_Birth           0
Country              0
Education            0
Marital_Status       0
 Income              0
Kidhome              0
Teenhome             0
Dt_Customer          0
Recency              0
MntWines             0
MntFruits            0
MntMeatProducts      0
MntFishProducts      0
MntSweetProducts     0
MntGoldProds         0
NumDealsPurchases    0
NumWebPurchases      0
NumStorePurchases    0
NumWebVisitsMonth    0
Complain             0
dtype: int64

In [141]:
df.head()

ID  Year_Birth Country   Education Marital_Status      Income   Kidhome  \
0   1826        1970      SP  Graduation       Divorced  $84,835.00         0   
1      1        1961      CA  Graduation         Single  $57,091.00         0   
2  10476        1958      US  Graduation        Married  $67,267.00         0   
3   1386        1967     AUS  Graduation       Together  $32,474.00         1   
4   5371        1989      SP  Graduation         Single  $21,474.00         1   

   Teenhome Dt_Customer  Recency  ...  MntFruits  MntMeatProducts  \
0         0     6/16/14        0  ...        104              379   
1         0     6/15/14        0  ...          5               64   
2         1     5/13/14        0  ...         11               59   
3         1     5/11/14        0  ...          0                1   
4         0      4/8/14        0  ...         16               24   

   MntFishProducts  MntSweetProducts  MntGoldProds  NumDealsPurchases  \
0              111               189           218                  1   
1                7                 0            37                  1   
2               15                 2            30                  1   
3                0                 0             0                  1   
4               11                 0            34                  2   

   NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  Complain  
0                4                  6                  1         0  
1                7                  7                  5         0  
2                3                  5                  2         0  
3                1                  2                  7         0  
4                3                  2                  7         0  

[5 rows x 21 columns]

In [142]:
# TotalMntProducts 
df['TotalMntProducts'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + df['MntSweetProducts'] + df['MntGoldProds']

In [143]:
df['TotalNumPurchases'] = df['NumDealsPurchases'] + df['NumWebPurchases'] + df['NumStorePurchases'] + df['NumWebVisitsMonth'] 

In [144]:
from datetime import datetime

In [145]:
today = datetime.today().strftime("%Y-%m-%d")
today = pd.to_datetime(today)
today

Timestamp('2022-06-30 00:00:00')

In [146]:
pd.to_datetime(today)

Timestamp('2022-06-30 00:00:00')

In [147]:
df["Dt_Customer"].astype("str")

0        6/16/14
1        6/15/14
2        5/13/14
3        5/11/14
4         4/8/14
          ...   
2211      3/7/13
2212     1/22/13
2213     12/3/12
2214    11/29/12
2215      9/1/12
Name: Dt_Customer, Length: 2216, dtype: object

In [148]:
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"])

In [149]:
df["Dt_Customer"]

0      2014-06-16
1      2014-06-15
2      2014-05-13
3      2014-05-11
4      2014-04-08
          ...    
2211   2013-03-07
2212   2013-01-22
2213   2012-12-03
2214   2012-11-29
2215   2012-09-01
Name: Dt_Customer, Length: 2216, dtype: datetime64[ns]

In [150]:
df["DaysSinceJoined"]  = today - df["Dt_Customer"]

In [151]:
df["DaysSinceJoined"] = df["DaysSinceJoined"].astype(str)
df.head()

ID  Year_Birth Country   Education Marital_Status      Income   Kidhome  \
0   1826        1970      SP  Graduation       Divorced  $84,835.00         0   
1      1        1961      CA  Graduation         Single  $57,091.00         0   
2  10476        1958      US  Graduation        Married  $67,267.00         0   
3   1386        1967     AUS  Graduation       Together  $32,474.00         1   
4   5371        1989      SP  Graduation         Single  $21,474.00         1   

   Teenhome Dt_Customer  Recency  ...  MntSweetProducts  MntGoldProds  \
0         0  2014-06-16        0  ...               189           218   
1         0  2014-06-15        0  ...                 0            37   
2         1  2014-05-13        0  ...                 2            30   
3         1  2014-05-11        0  ...                 0             0   
4         0  2014-04-08        0  ...                 0            34   

   NumDealsPurchases  NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  \
0                  1                4                  6                  1   
1                  1                7                  7                  5   
2                  1                3                  5                  2   
3                  1                1                  2                  7   
4                  2                3                  2                  7   

   Complain  TotalMntProducts  TotalNumPurchases  DaysSinceJoined  
0         0              1190                 12        2936 days  
1         0               577                 20        2937 days  
2         0               251                 11        2970 days  
3         0                11                 11        2972 days  
4         0                91                 14        3005 days  

[5 rows x 24 columns]

In [152]:
df["DaysSinceJoined"] = df["DaysSinceJoined"].replace('days','')
df.head()

ID  Year_Birth Country   Education Marital_Status      Income   Kidhome  \
0   1826        1970      SP  Graduation       Divorced  $84,835.00         0   
1      1        1961      CA  Graduation         Single  $57,091.00         0   
2  10476        1958      US  Graduation        Married  $67,267.00         0   
3   1386        1967     AUS  Graduation       Together  $32,474.00         1   
4   5371        1989      SP  Graduation         Single  $21,474.00         1   

   Teenhome Dt_Customer  Recency  ...  MntSweetProducts  MntGoldProds  \
0         0  2014-06-16        0  ...               189           218   
1         0  2014-06-15        0  ...                 0            37   
2         1  2014-05-13        0  ...                 2            30   
3         1  2014-05-11        0  ...                 0             0   
4         0  2014-04-08        0  ...                 0            34   

   NumDealsPurchases  NumWebPurchases  NumStorePurchases  NumWebVisitsMonth  \
0                  1                4                  6                  1   
1                  1                7                  7                  5   
2                  1                3                  5                  2   
3                  1                1                  2                  7   
4                  2                3                  2                  7   

   Complain  TotalMntProducts  TotalNumPurchases  DaysSinceJoined  
0         0              1190                 12        2936 days  
1         0               577                 20        2937 days  
2         0               251                 11        2970 days  
3         0                11                 11        2972 days  
4         0                91                 14        3005 days  

[5 rows x 24 columns]

In [153]:
condition = df["TotalMntProducts"] >= 2000 
# df['VIP']
df_vip = df[condition]
df_vip.head()

ID  Year_Birth Country   Education Marital_Status      Income   \
70    1772        1975      CA         PhD        Married  $79,174.00    
77    7919        1976      SP         PhD       Together  $72,335.00    
281   3403        1958      SP         PhD       Together  $78,952.00    
333   3698        1983      SP  Graduation       Together  $78,687.00    
374  10133        1970      CA  Graduation         Single  $93,790.00    

     Kidhome  Teenhome Dt_Customer  Recency  ...  MntSweetProducts  \
70         0         0  2013-01-11        2  ...                92   
77         0         0  2012-08-13        2  ...                21   
281        0         1  2012-09-24       11  ...                76   
333        0         0  2012-08-09       13  ...                55   
374        0         0  2014-02-12       16  ...               114   

     MntGoldProds  NumDealsPurchases  NumWebPurchases  NumStorePurchases  \
70            129                  1                5                  7   
77              0                  1               10                  8   
281           134                  2                2                 12   
333           241                  1                4                  8   
374            45                  0                6                 12   

     NumWebVisitsMonth  Complain  TotalMntProducts  TotalNumPurchases  \
70                   2         0              2043                 15   
77                   8         0              2092                 27   
281                  6         0              2092                 22   
333                  2         0              2130                 15   
374                  2         0              2349                 20   

     DaysSinceJoined  
70         3457 days  
77         3608 days  
281        3566 days  
333        3612 days  
374        3060 days  

[5 rows x 24 columns]

## Part 2:



[공식문서](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html)를 참고하여 **최소 2개**의 two-sample $\chi^2$ 독립성 검정을 시행한 후, 그 결과를 해석하세요

#### Case 1)

In [154]:
# 이 곳에 답안을 작성하십시오

** **`해석은 여기에 작성하세요`** **

#### Case 2)

In [155]:
# 이 곳에 답안을 작성하십시오

** **`해석은 여기에 작성하세요`** **